In [1]:
# check the number of workers for concurrent jobs
!nproc

12


In [2]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 22.2 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:

# try usual models and select the best one
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score


from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
from typing import List, Tuple, Dict, Any
import os
import sys
import time
import warnings

warnings.filterwarnings("ignore")

current_dir = "/content/drive/MyDrive/ETH/task-2-ECG-Signals-Classification/Models"
print(current_dir)
sys.path.append(current_dir)
from MODELS import finetuned_models as models
from MODELS import vanilla_models as vanilla_models
sys.path.append(current_dir + "/../")
sys.path.append(current_dir + "/../FeaturePreprocessing/")

from utils import *
from FeaturePreprocessing.process import *


current_dir = "/content/drive/MyDrive/ETH/task-2-ECG-Signals-Classification/Models"
print(current_dir)

/content/drive/MyDrive/ETH/task-2-ECG-Signals-Classification/Models
/content/drive/MyDrive/ETH/task-2-ECG-Signals-Classification/FeaturePreprocessing
/content/drive/MyDrive/ETH/task-2-ECG-Signals-Classification/Models


In [10]:
X_train_p1 = pd.read_csv(current_dir + "/../Data/final/p1_X_train.csv", index_col=0)
X_test_p1 = pd.read_csv(current_dir + "/../Data/final/p1_X_test.csv", index_col=0)
y_train_p1 = pd.read_csv(current_dir + "/../Data/final/p1_y_train.csv", index_col=0)
X_train_p2 = pd.read_csv(current_dir + "/../Data/final/p2_X_train.csv", index_col=0)
X_test_p2 = pd.read_csv(current_dir + "/../Data/final/p2_X_test.csv", index_col=0)
y_train_p2 = pd.read_csv(current_dir + "/../Data/final/p2_y_train.csv", index_col=0)
print(X_train_p1.shape, X_test_p1.shape, y_train_p1.shape)
print(X_train_p2.shape, X_test_p2.shape, y_train_p2.shape)
assert X_train_p1.shape[0] == y_train_p1.shape[0], "X_train_p1 and y_train_p1 not equal"
assert X_train_p2.shape[0] == y_train_p2.shape[0], "X_train_p2 and y_train_p2 not equal"
assert (y_train_p2.values.ravel() == y_train_p2.values.ravel()).all() == True, "y_train_p2 not equal to y_train_p2"
X_train_total = pd.concat([X_train_p1, X_train_p2], axis=1) # concat along columns, column-wise
X_test_total = pd.concat([X_test_p1, X_test_p2], axis=1)
y_train_total = y_train_p1
def check_inf(X):
    inf_col = set()
    for col in X.columns:
        if np.isinf(X[col]).sum() > 0:
            print(col, np.isinf(X[col]).sum())
            inf_col.add(col)
    return inf_col
inf_col = check_inf(X_train_total) | check_inf(X_test_total)
inf_col = list(inf_col)
#drop the columns with inf values
X_train_total = X_train_total.drop(inf_col, axis=1)
X_test_total = X_test_total.drop(inf_col, axis=1)
X_train_total.to_csv(current_dir + "/../Data/final/X_train_total.csv")
X_test_total.to_csv(current_dir + "/../Data/final/X_test_total.csv")
y_train_total.to_csv(current_dir + "/../Data/final/y_train_total.csv")
print(X_train_total.shape, X_test_total.shape, y_train_total.shape)

(5117, 523) (3411, 523) (5117, 1)
(5117, 2239) (3411, 2239) (5117, 1)
hc_317 2
hc_422 2
hc_436 251
hc_442 1
hc_317 1
hc_422 4
hc_436 136
(5117, 2758) (3411, 2758) (5117, 1)


In [11]:
# check whether X_train_total has inf
inf_col = check_inf(X_train_total) | check_inf(X_test_total)
inf_col = list(inf_col)
print(inf_col)

[]


In [15]:
from math import e
# use select k best to select the best features
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('selector', SelectKBest(f_classif, k=290)),
])

pipe.fit(X_train_total, y_train_total.values.ravel())
X_train = pipe.transform(X_train_total)
X_test = pipe.transform(X_test_total)
y_train = y_train_total.values.ravel()
print(X_train_total.shape, X_test_total.shape, y_train_total.shape)
print(X_train.shape, X_test.shape, y_train.shape)

# use xgboosting + lightgbm to test the best chocie of K
# use cross validation
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score

xgb = models["XGB"]
# set to gpu
xgb.set_params(tree_method="gpu_hist")
lgbm = models["LGBM"]

# f1_xgb = cross_val_score(xgb, X_train, y_train, cv=5, scoring=make_scorer(f1_score, average="micro"), n_jobs=-1, verbose=0).mean()



# X_tr, X_val, y_tr, y_val = train_test_split(X_train_total, y_train_total, test_size=0.2, random_state=42, stratify=y_train_total)
# print(X_tr.shape, X_val.shape, y_tr.shape, y_val.shape)

# xgb.fit(X_tr, y_tr)
# lgbm.fit(X_tr, y_tr)

# y_pred_xgb = xgb.predict(X_val)
# y_pred_lgbm = lgbm.predict(X_val)

# f1_xgb = f1_score(y_val, y_pred_xgb, average="micro")
# f1_lgbm = f1_score(y_val, y_pred_lgbm, average="micro")

# try cross_val
# cross validation


# f1_lgbm = cross_val_score(lgbm, X_train_total, y_train_total.values.ravel(), cv=5, scoring=make_scorer(f1_score, average="micro"), n_jobs=-1, verbose=2)

# print("F1 score of xgboosting: ", f1_xgb)
# print("F1 score of lightgbm: ", f1_lgbm)

xgb.fit(X_train_total, y_train_total.values.ravel())
y_pred_xgb = xgb.predict(X_test_total)
submit_pred(y_pred_xgb, "vanilla_xgb_2758.csv")
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)
submit_pred(y_pred_xgb, "vanilla_xgb_290.csv")

(5117, 2758) (3411, 2758) (5117, 1)
(5117, 290) (3411, 290) (5117,)


In [ ]:
f1_xgb.mean()

0.888018351356305

In [ ]:
from sklearn.ensemble import VotingClassifier

models = vanilla_models
X_train, X_test, y_train = X_train_total, X_test_total, y_train_total.values.ravel()
print(X_train.shape, X_test.shape, y_train.shape)

with open(os.path.join(current_dir, "selected_models.txt"), "r") as f:
    scores = f.read()
    scores = [score.split("|") for score in scores.split("\n") if score != ""]
    scores = {
        score[0]: float(score[1]) for score in scores if float(score[1]) > 0.86
    }

print("Scores: ", scores)
s = [scores[name] for name in scores.keys()]
weights = calculate_weights(s)
print("Weights: ", weights)


models["XGB"].set_params(tree_method="gpu_hist")

# 确保 estimators 和 weights 的模型名称一致
estimators = [(name, models[name]) for name in scores.keys()]
voting = VotingClassifier(estimators=estimators, voting="soft", weights=weights)

print("Train the model...", estimators)

@function_runtime_tracker
def train(model):
    # train on the whole dataset
    model.fit(X_train, y_train)
    return model

voting = train(voting)
# predict on the test set
y_pred = voting.predict(X_test)
submit_pred(y_pred, "vanilla_p1Andp2_2758.csv")

print("Trying models...")

(5117, 2758) (3411, 2758) (5117,)
Scores:  {'LGBM': 0.8761, 'XGB': 0.8761, 'CatBoost': 0.876, 'GradientBoosting': 0.8701, 'ExtraTrees': 0.8799, 'Bagging': 0.8633, 'RandomForest': 0.873}
Weights:  [0.1591259  0.1591259  0.15825421 0.11444702 0.19606222 0.07877407
 0.13421068]
Train the model... [('LGBM', LGBMClassifier(n_jobs=-1, random_state=42, verbose=-1)), ('XGB', XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
 

In [ ]:
X_train_p2

,ts_mean,ts_median,ts_std,ts_var,ts_max,ts_rms,ts_peak_to_peak,ts_skewness,ts_kurtosis,ts_sma,...,psd_heart_rate_db4_coeff_3_skewness,psd_heart_rate_db4_coeff_3_kurtosis,psd_heart_rate_db4_coeff_3_zero_crossing_rate,psd_heart_rate_db4_coeff_3_sma,psd_heart_rate_db4_coeff_3_energy,psd_heart_rate_db4_coeff_3_entropy,psd_heart_rate_db4_coeff_3_crest_factor,psd_heart_rate_db4_coeff_3_impulse_factor,psd_heart_rate_db4_coeff_3_shape_factor,psd_heart_rate_db4_coeff_3_clearance_factor
0,2.358328,2.358328,2.358328,2.498459,2.358328,2.358328,2.358328,-1.898557,0.747787,2.498463,...,0.728193,2.127665,1.500865,-0.317260,-0.115568,0.0,1.882603,-0.003167,1.834877,1.882603
1,-0.159019,-0.159019,-0.159019,-0.259136,-0.159019,-0.159019,-0.159019,0.422035,0.213294,-0.259140,...,-0.496195,-1.224862,-0.131676,-0.304991,-0.115443,0.0,-1.759056,-0.003167,-0.299783,-1.759056
2,-0.265300,-0.265300,-0.265300,-0.337662,-0.265300,-0.265300,-0.265300,0.295985,0.153356,-0.337665,...,0.173624,-0.294794,0.521340,-0.353000,-0.115931,0.0,0.468888,-0.003167,-0.625273,0.468888
3,-0.105879,-0.105879,-0.105879,-0.218721,-0.105879,-0.105879,-0.105879,0.152473,0.240758,-0.218726,...,2.597072,4.787647,0.847849,0.159221,-0.075958,0.0,3.019223,-0.003167,3.315027,3.019223
4,-0.188506,-0.188506,-0.188506,-0.281230,-0.188506,-0.188506,-0.188506,0.706163,0.197361,-0.281234,...,-1.086122,1.466217,0.521340,-0.107121,-0.107562,0.0,0.355088,-0.003167,1.023003,0.355088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5112,-1.255195,-1.255195,-1.255195,-0.921586,-1.255195,-1.255195,-1.255195,-2.432225,-1.066471,-0.921570,...,-0.366102,-1.045898,-1.437709,2.850357,1.013411,0.0,-1.433959,-0.003167,-0.709154,-1.433959
5113,-0.253311,-0.253311,-0.253311,-0.328958,-0.253311,-0.253311,-0.253311,1.416197,0.160476,-0.328961,...,1.275293,-0.688056,-0.131676,-0.382121,-0.116015,0.0,0.575874,-0.003167,0.475115,0.575874
5114,-0.184617,-0.184617,-0.184617,-0.278330,-0.184617,-0.184617,-0.184617,-0.275849,0.199492,-0.278334,...,-0.745051,-0.819839,-0.131676,0.277224,-0.070315,0.0,-1.417905,-0.003167,0.062778,-1.417905
5115,-0.228037,-0.228037,-0.228037,-0.310480,-0.228037,-0.228037,-0.228037,0.336433,0.175177,-0.310483,...,-0.123430,-1.095721,-0.131676,-0.376254,-0.116011,0.0,-1.000472,-0.003167,-1.072357,-1.000472
